<a href="https://colab.research.google.com/github/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/Chapter_8_Using_Tensorflow_to_Create_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 8: Using Tensorflow to Create Text  
### Hi, chào mọi người, tụi mình lại gặp nhau rồi 🌻.  
### Trong chương này tụi mình sẽ tìm hiểu về cách sử dụng các mô hình để tạo ra văn bản. Nói nôm na là giống như cách mà các chatbot hiện tại tạo ra câu trả lời cho mọi người á. Nhưng cái này ở cấp độ sơ khai hơn. 😊  

![predictext](https://cgupta.tech/images/rnn_representative.png)

Như mọi người thấy ở bức hình trên, đây là cơ chế chính của tính năng tạo văn bản trong chương này.  

Trước khi đi vào phân tích nó thì chúng ta cùng ôn lại các chương trước tí nhá. 😊  

---

- Ở **chương 5**, mọi người tìm hiểu về cơ chế **Tokenize** để tách văn bản, tạo từ điển, mã hóa ở cấp từ hay ký tự rồi thêm bộ đệm hay padding vào.  

- Sang đến **chương 6**, chúng ta tìm hiểu về cơ chế **Embedding - Vector biểu diễn** trên các chiều không gian cao hơn.  

- Và gần đây nhất là **chương 7**, tìm hiểu về mạng hồi quy **RNN** và lớp **LSTM** qua đó cải thiện khả năng hiểu ngữ nghĩa theo trình tự của chuỗi.  

Bây giờ đến với **chương 8**, tụi mình sẽ tìm hiểu về cơ chế **tạo/sinh văn bản** của các mô hình. Thật ra nó không khó như mọi người nghĩ mà rất là đơn giản. Chúng đều dựa trên những cơ chế trước giờ chúng ta học, tuy nhiên khác ở khâu xử lý và biến đổi dữ liệu tí.  

Vậy sự khác nhau này là gì, tụi mình sẽ cùng tìm hiểu nhá. 🌟  

![process_text](https://i.imgur.com/7kOLuRM.png)

![padding_text](https://i.imgur.com/QOcbcqC.png)

Trong các bài toán trước đây của chúng ta, tụi mình sẽ tiến hành **xử lý dữ liệu văn bản sang dạng số** và **phân loại chúng** dựa trên **nhãn có sẵn** của dữ liệu huấn luyện trước đó. 📊  

Thường thì **số lượng nhãn khá ít**, ví dụ như chỉ có hai nhãn **"tích cực"** và **"tiêu cực"** trong bài toán phân tích cảm xúc, hoặc vài nhãn cụ thể trong bài toán phân loại văn bản. 🏷️

![classify_sentiment](https://media.mlhive.com/i/max/YbAcdK8b2Q4T5Ed4fIf.png)

Giờ đây đến với **bài toán tạo/sinh văn bản**, thay vì **phân loại với các nhãn cố định**, chúng ta sẽ **dự đoán từ tiếp theo** của văn bản đó. 📝✨  

> **Ơ, vậy nhãn của các văn bản sẽ chính là từ tiếp theo của chúng à?**  

Yeah, chính xác rồi đó! 🌟

![example](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_8/input_labels.png?raw=true)

> **Ôi, thế số lượng nhãn của mô hình chẳng phải sẽ rất khổng lồ hay sao （⊙ｏ⊙）?**  

Yeah, bạn lại đoán đúng rồi. 🎉  

Số lượng nhãn chính là **số từ trong bộ từ điển** á =))  
Có khi nó lên đến tận **10.000** và thường số lượng epochs huấn luyện cũng rất lớn, có thể từ **1.000** trở lên.  

> **Vậy chúng tạo ra văn bản như thế nào?**  

Đơn giản thôi, tụi mình sẽ tiến hành **lặp đi lặp lại quá trình dự đoán từ tiếp theo**. 🔄  
Khi mọi người dự đoán được một từ mới, chúng ta sẽ lấy từ đó **kết hợp lại với văn bản đầu vào** và tiếp tục dự đoán từ kế tiếp.  
Cứ tiến hành như thế cho đến khi hết số vòng lặp hoặc đạt giới hạn số lượng từ dự đoán. 🌟  

Nó giống hệt với hình mà tụi mình đã nhìn thấy ở phần đầu đó. 📄✨  

![predictext](https://cgupta.tech/images/rnn_representative.png)

Ở đây, câu văn ban đầu của chúng ta là: **"the man is walking"**, phần màu xanh dương là **cửa sổ trượt - window data** để cắt lấy dữ liệu đầu vào với độ dài tối đa hiện tại là **4 từ**. Do đó, **dữ liệu đầu vào** của chúng ta sẽ là **"the man is walking"**.  

***Mọi người nhớ kĩ phần cửa sổ trượt cắt lấy dữ liệu đầu vào giúp mình nha.*** 📝  

Quá trình tạo câu văn, ở đây mình cho số lượng từ tạo ra là **4**, nên chúng ta sẽ có **4 lần lặp**.  

**Step 1**: Nhận đầu vào là **"the man is walking"**, tụi mình dự đoán ra được từ **"down"**.  

**Step 2**: Tụi mình kết hợp từ **"down"** vào chung với văn bản đầu vào lúc trước là **"the man is walking"** thành **"the man is walking down"**, sau đó **cửa sổ trượt với kích thước là 4** sẽ cắt lại dữ liệu đầu vào thành **"man is walking down"** và dự đoán ra từ **"the"**.  

**Step 3**: Tương tự như trước, dữ liệu đầu vào sau khi cắt là **"is walking down the"**, dự đoán ra từ **"street"**.  

**Step 4**: Dữ liệu đầu vào là **"walking down the street"**, dự đoán ra dấu **"."**.  

Vậy câu cuối cùng chúng ta dự đoán ra được sẽ là:  
> **"the man is walking down the street ."** 🚶‍♂️✨  

# Lý thuyết đủ nhiều rồi, tụi mình sẽ tiến hành đi vào thực hành luôn nha. ヾ(•ω•`)o

In [ ]:
import time
import kagglehub
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from datetime import timedelta
import matplotlib.pyplot as plt

In [ ]:
# Tạo một MirroredStrategy để sử dụng tất cả các GPU có sẵn
strategy = tf.distribute.MirroredStrategy()

print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# Tiến hành xử lý dữ liệu 🛠️✨

Dữ liệu mà chúng ta sẽ dùng là đoạn thơ dưới đây nha.

In [ ]:
data = """In the town of Athy one Jeremy Lanigan
 Battered away til he hadnt a pound.
 His father died and made him a man again
 Left him a farm and ten acres of ground.
 He gave a grand party for friends and relations
 Who didnt forget him when come to the wall,
 And if youll but listen Ill make your eyes glisten
 Of the rows and the ructions of Lanigan’s Ball.
 Myself to be sure got free invitation,
 For all the nice girls and boys I might ask,
 And just in a minute both friends and relations
 Were dancing round merry as bees round a cask."""

In [ ]:
# Kiểm tra cú pháp đoạn thơ
print(repr(data))

In [ ]:
# Tụi mình sẽ coi mẫu dòng trong đoạn thơ là một câu và tách chúng ra dựa trên dấu xuống hàng \n nha
corpus = data.lower().split("\n")

# Tạo bộ từ điển từ bộ dữ liệu của chúng ta.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

# Kiểm tra bộ từ điển
print(tokenizer.word_index)
print(f"Số lượng từ trong bộ từ điển là: {len(tokenizer.word_index)}")

In [ ]:
total_words = len(tokenizer.word_index) + 1 # Thêm 1 vô đại diện cho phần token <padding>: 0

### Bây giờ tụi mình sẽ tới với phần xử lý dữ liệu huấn luyện. 🛠️  
***Mọi người chú ý kĩ phần này, tránh nhầm lẫn giúp mình nha. Ở phần thực hành này, tạm thời chúng ta sẽ không sử dụng cửa sổ trượt!***  

Đầu tiên, ta tiến hành mã hóa câu từ dữ liệu văn bản sang chuỗi dạng số. 🔢

![encode](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_8/encode.png?raw=true)

Tiếp đến, chúng ta sẽ tách chuỗi thành nhiều chuỗi con nhỏ hơn với độ dài chuỗi tăng dần. ✂️  

***Chú ý: Mọi người cần phải phân biệt, hiện tại chúng ta không sử dụng cửa sổ trượt ở đây nha!*** 🧐  

![sub_sequences](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_8/sub_sequences.png?raw=true)

Như mọi người thấy, từ một chuỗi ban đầu chúng ta đã tách ra thành 7 chuỗi con, nhờ đó gia tăng được số lượng dữ liệu. 🌱 Các chuỗi con để huấn luyện này còn có tên gọi khác là **"seed text - hạt giống văn bản"**, khởi nguồn để nảy mầm, tạo nên các văn bản dự đoán sau này.
Việc chia các chuỗi con này giúp cho mô hình có thể học được ngữ cảnh ở nhiều độ dài khác nhau.🌟

In [ ]:
input_sequences = []
for line in corpus:
  # Mã hóa câu (line) thành chuỗi số
  token_list = tokenizer.texts_to_sequences([line])[0] # Vì dữ liệu trả về của hàm là dạng một list nên để lấy được chuỗi số phải lấy phần tử 0 của list
  for i in range(1, len(token_list)):
    # Tiến hành tách ra thành từng chuỗi con
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

# Kiểm tra thử 5 chuỗi đầu tiên
for i in range(5):
  print(input_sequences[i])

Bây giờ chúng ta tiến hành **padding** cho dữ liệu dựa trên độ dài của câu dài nhất. 🛠️

Lý do là bởi vì không cố định kích thước như **cửa sổ trượt**, dữ liệu giữa các câu của chúng ta có kích thước chênh lệch rất nhiều. Chúng ta cần **đồng dạng kích thước** để đưa vào huấn luyện mô hình. 📏

*💡 Lưu ý: Trong trường hợp này, chúng ta sẽ **padding ở đầu** (pre-padding) nha, để từ có thể ảnh hưởng mạnh hơn đến từ dự đoán phía sau, giống với **khả năng ảnh hưởng của trình tự các từ** mà chúng ta đã tìm hiểu ở chương 7.* 🌟

![padding_sequences](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_8/pad_sequences.png?raw=true)


In [ ]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences_padded = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding="pre"))

# Chỉnh lại số lượng token vì đã thêm token padding vào <pad>: 0
total_words = len(tokenizer.word_index) + 1

# Kiểm tra chuỗi sau khi đệm
for i in range(5):
  print(input_sequences_padded[i])

Vậy là các chuỗi của chúng ta đã được **đệm đầy đủ** với kích thước là **10**. 🎉

Bây giờ, chúng ta sẽ tiến hành **chia dữ liệu** thành hai phần:

1. **Dữ liệu huấn luyện**: Chính là các từ trong chuỗi **trừ từ cuối cùng**.
2. **Nhãn**: Là **từ cuối cùng** của mỗi chuỗi.

Quá trình này sẽ giúp chúng ta có dữ liệu phù hợp để huấn luyện mô hình dự đoán từ tiếp theo. 🚀

![padding_sequences](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_8/x_labels.png?raw=true)

In [ ]:
xs = input_sequences_padded[:,:-1]
labels = input_sequences_padded[:,-1]

Ở đây tụi mình sẽ áp dụng một **thủ thuật nhỏ** nhé! 🌟 Đây cũng là cơ hội để tiếp cận một loại **loss function** khác.

Như trước đây mình đã đề cập, có sự khác biệt giữa **`sparse_categorical_crossentropy`** và **`categorical_crossentropy`**:

- Nếu dữ liệu nhãn là dạng **số index** (ví dụ: `[0, 1, 2]`), ta sẽ sử dụng **`sparse_categorical_crossentropy`**.
- Trong trường hợp này, tụi mình sẽ tiến hành **mã hóa nhãn theo one-hot-encoding**, nên cần sử dụng **`categorical_crossentropy`**.

### Lưu ý nhỏ:
- Sử dụng **index-based labels** (dạng số) có thể **tiết kiệm bộ nhớ hơn** vì không cần lưu trữ toàn bộ vector one-hot-encoding.
- Tuy nhiên, vì sách đã hướng dẫn như vậy, tụi mình cứ làm theo để **làm quen với cả hai cách** nhé! ✨

![dataset](https://github.com/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/images/Chapter_8/dataset.png?raw=true)

In [ ]:
# Tiến hành mã hóa one-hot-encoding cho nhãn
from tensorflow.keras.utils import to_categorical
ys = to_categorical(labels, num_classes = total_words)

In [ ]:
# Kiểm tra lại dữ liệu của chúng ta
print(f"Chuỗi được mã hóa: {input_sequences[0]}")
print(f"Chuỗi x để huấn luyện: {xs[0]}")
print(f"Nhãn của chuỗi x, tức từ tiếp theo: {labels[0]}")
print(f"Nhãn sau khi one-hot-encoding:\n {ys[0]}")

# Tiến hành tạo và huấn luyện mô hình 🚀

Ở đây tụi mình sẽ thử khởi tạo một mô hình đơn giản thôi nha.  

Số chiều **Embedding** tụi mình tạm cho là 8 nha vì kích thước từ điển, số lượng từ cũng khá nhỏ.  

Về tham số trong lớp **BLSTM**, số đơn vị ẩn như chương trước tụi mình để theo số chiều Embedding thì ở đây mình để tương đương với độ dài của chuỗi ban đầu trừ đi 1 nha (vì chúng ta đã lấy token cuối làm nhãn nên trừ 1 đi).  

Đầu ra lớp tuyến tính - **Dense**, thì sẽ bằng với tổng số lượng tokens hay số lượng từ với 1 token padding. ✨

In [ ]:
# Xây dựng kiến trúc mô hình
model = Sequential([
    Embedding(total_words, 8),
    Bidirectional(LSTM(max_sequence_len-1)),
    Dense(total_words, activation = 'softmax')
])

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

Bởi vì mô hình ở đây khá đơn giản và ít dữ liệu nên chúng ta tiến hành huấn luyện lâu hơn với **số epoch là 15.000**.

In [ ]:
import time
start_time = time.time()
history = model.fit(xs, ys, epochs=1500)
end_time = time.time()

In [ ]:
print(f"Thời gian huấn luyện: {timedelta(seconds=end_time - start_time)} giây")

In [ ]:
# Vẽ biểu đồ huấn luyện
import matplotlib.pyplot as plt

train_acc = history.history["accuracy"]
train_loss = history.history["loss"]

fig, axs = plt.subplots(1, 2, figsize = (12, 4))
axs[0].plot(train_acc)
axs[0].set_title("Training Accuracy")
axs[1].plot(train_loss, color="orange")
axs[1].set_title("Training Loss")
plt.tight_layout()
plt.show()

In [ ]:
eval = model.evaluate(xs, ys)

In [ ]:
model.save("model.h5")

In [ ]:
# Tải lại model
model = load_model("model.h5")

Kết quả cuối cùng tụi mình đo được sẽ khoảng cỡ 95%, điều đó có nghĩa là từ dự đoán được tiếp theo có xác suất 95% giống với từ đã được huấn luyện bởi chuỗi trước đó.  

***Mọi người lưu ý giúp mình: Ở đây sử dụng độ chính xác - accuracy để đo không thật sự hiệu quả đối với bài toán ngữ nghĩa tạo/sinh văn bản như này. Cho đến thời điểm hiện tại tụi mình học, vẫn chưa có thang đo nào phù hợp tuyệt đối cho bài toán.*** 🌟

### Predicting the Next Word  
### Bây giờ tụi mình sẽ tiến hành dự đoán thử một từ tiếp theo trước nha.  

Văn bản mình chọn là **"in the town of athy"** nằm ở câu đầu tiên của dữ liệu huấn luyện nha. 🌟  

In [ ]:
seed_text =  "in the town of athy"

Đầu tiên ta sẽ tiến hành tokenize cho chuỗi đó, sau đó thì padding và đưa vào mô hình

In [ ]:
token_list = tokenizer.texts_to_sequences([seed_text])[0]
token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre') # -1 vào max_sequence_len bởi vì chuỗi này không có nhãn
# Tiến hành dự đoán
predicted = model.predict(token_list, verbose=1)
predicted_index = np.argmax(predicted, axis = -1)

In [ ]:
# Tạo index to word để chuyển từ dữ liệu dạng số sang văn bản lại
index_words ={}
for word, index in tokenizer.word_index.items():
    index_words[index] = word

# Chuyển index dự đoán thành dạng chữ
predicted_word = index_words[predicted_index[0]]

print(f"Chuỗi ban đầu: {seed_text}")
print(f"Từ dự đoán: '{predicted_word}' với sác xuất {predicted[0][predicted_index[0]]*100:.2f}%")

Uầy ra từ "one" đúng với câu đầu tiên trong dữ liệu huấn luyện luôn này.
> In the town of Athy one Jeremy Lanigan<br>
 Battered away til he hadnt a pound.

### Compounding Predictions to Generate Text  
### Bây giờ tụi mình sẽ tiến hành tạo/sinh văn bản dựa trên workflow trước đó tụi mình định nghĩa nha. 🌟✨🌈  

***Lưu ý: Ở đây chúng ta chọn padding chuỗi theo độ dài câu dài nhất vì ban đầu khi dữ liệu huấn luyện cũng được làm như vậy chứ chưa sử dụng phương pháp cửa sổ trượt nha. Để mà nói mọi người cũng có thể hiểu trường hợp này kích thước cửa sổ trượt là độ dài câu dài nhất cũng được. Từ đó các chuỗi không đủ kích thước thì sẽ padding thêm vào. 🔄📏  ***

Thông qua việc lặp đi lặp lại quá trình dự đoán từ tiếp theo, kết hợp lại với dữ liệu đầu vào rồi dự đoán tiếp ta sẽ tạo/sinh ra được phần tiếp theo cho câu văn. 📝🔮  

Ở đây mình sử dụng dữ liệu đầu vào ban đầu là **"sweet jeremy saw dublin"** nha. Tụi mình sẽ tiến hành dự đoán 10 từ tiếp theo phía sau. 🎯📖

In [ ]:
seed_text = "sweet jeremy saw dublin"
predicted_text = seed_text
n_words = 10

for i in range(n_words):
    token_list = tokenizer.texts_to_sequences([predicted_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=1)
    predicted_index = np.argmax(predicted, axis=-1)
    output_word = index_words[predicted_index[0]]

    print(f"Step {i}:")
    print(f"Văn bản đầu vào: {predicted_text}")
    print(f"Chuỗi mã hóa đầu vào: {token_list}")
    print(f"Từ dự đoán: {output_word} với xác suất {predicted[0][predicted_index[0]]*100:2f}")
    print("-"*50)
    predicted_text += " " + output_word

print(predicted_text)


Đây là kết quả ở lần chạy của mình nha:  

> "sweet jeremy saw dublin all the rows a and of lanigan’s ball ask ask"  

Mọi người có thể thấy càng về sau thì các từ tạo ra khiến câu văn càng trở nên vô nghĩa hơn, thậm chí sai cả về mặt ngữ pháp của từ. 😅📉  

> **Vậy lỗi này do đâu mà ra?**  

Có 2 lý do chính ảnh hưởng đến việc này là:  

1. **Dữ liệu huấn luyện của chúng ta quá nhỏ** khiến mô hình không thể nắm bắt được nhiều ngữ cảnh. 📚❌  
2. **Hiệu ứng dây chuyền, cánh bướm hay hiệu ứng Domino.** Khi một việc gì đó xảy ra thì các sự việc sau phụ thuộc vào nó sẽ chịu ảnh hưởng lớn dần lên. Ở đây, khi một từ được dự đoán không tốt, các từ phía sau nó lại được dự đoán dựa trên nó và các từ phía trước cũng sẽ trở nên tệ hơn. Dần dần sự tệ hại này ngày một lớn dần khiến cho các từ nối vào câu văn càng về sau trở nên vô nghĩa hơn rất là nhiều. 🔗🌀  

Có một fact là vào năm 2016 đã từng có một bộ phim viễn tưởng [Sunspring](https://en.wikipedia.org/wiki/Sunspring) được tạo ra bởi AI. Bộ phim này khá buồn cười, ban đầu nội dung vẫn ổn cho đến càng về sau chúng lại trở nên khó hiểu đúng nghĩa ba chấm luôn ý. 🎥🤖✨

# Extending the dataset  
### Bây giờ tụi mình sẽ làm việc với mô hình với bộ dữ liệu lớn hơn nha.  

Ở đây theo tác giả thì bộ dữ liệu dưới đây bao gồm khoảng 1.700 về lời các bài hát. Vì đường dẫn trong sách không thể sử dụng được nữa nên tụi mình sẽ tải dữ liệu từ Kaggle nha. 🎶📥  

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jamzhu/irishlyricseof")

print("Path to dataset files:", path)

In [ ]:
# # Liệt kê các file bên trong thư mục
import os
for f in os.listdir(path):
  print(f)

In [ ]:
# Tiến hành tải bộ dữ liệu
data = open(os.path.join(path,"irish-lyrics-eof.txt")).read()
corpus = data.lower().split("\n")
# In thử 5 hàng đầu tiên ra
for i in range(5):
  print(repr(corpus[i]))

In [ ]:
# Tiến hành tokenize dữ liệu
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1 # Thêm một cho token padding sau này
print(f"Tổng số lượng từ: {total_words}")
print(f"Bộ từ điển từ vựng: {tokenizer.word_index}")

In [ ]:
# Tạo bộ từ điển chuyển dữ liệu số thành chữ lại
index_words ={}
for word, index in tokenizer.word_index.items():
    index_words[index] = word

# Tiến hành tạo các seed text hay chuỗi con
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)-1):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Kiểm tra 5 chuỗi mã hóa số đầu tiên
for i in range(5):
  print(input_sequences[i])


In [ ]:
# Tiến hành padding cho dữ liệu
max_sequence_len = max([len(x) for x in input_sequences])
print(f"Độ dài chuỗi dài nhất: {max_sequence_len}")
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
# In thử 5 chuỗi đầu tiên
for i in range(5):
  print(input_sequences[i])

In [ ]:
# Tiến hành chia dữ liệu huấn luyện và nhãn
xs = input_sequences[:,:-1]
labels = input_sequences[:,-1]

In [ ]:
# Thiết lập mô hình
model1 = Sequential([
    Embedding(total_words, 8),
    Bidirectional(LSTM(max_sequence_len-1)),
    Dense(total_words, activation = 'softmax')
])

model1.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
start_time = time.time()
history1 = model1.fit(xs, labels, epochs=1000)
end_time = time.time()

In [ ]:
print(f"Thời gian huấn luyện: {timedelta(seconds=end_time-start_time)}")

In [ ]:
# Vẽ biểu đồ quá trình huấn luyện
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1, 2, figsize = (12, 4))
train_acc = history1.history["accuracy"]
train_loss = history1.history["loss"]
axs[0].plot(train_acc)
axs[0].set_title("Training Accuracy")
axs[1].plot(train_loss, color="orange")
axs[1].set_title("Training Loss")
plt.tight_layout()
plt.show()

In [ ]:
model1.evaluate(xs,labels)

In [ ]:
model1.save("model1.h5")

In [ ]:
# Tải lại mô hình
model1 = load_model("model1.h5")

Uầy, vậy là với bộ dữ liệu này, độ chính xác trên tập huấn luyện đạt được là khoảng 60%. 🎉

Bây giờ tụi mình sẽ tiến hành dự đoán thử với các vế trước đó nha. 🔍✨

In [ ]:
# Dự đoán một từ
seed_text = "in the town of athy"
token_list = tokenizer.texts_to_sequences([seed_text])[0]
token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre') # -1 vào max_sequence_len bởi vì chuỗi này không có nhãn
# Tiến hành dự đoán
predicted = model1.predict(token_list, verbose=1)
predicted_index = np.argmax(predicted, axis = -1)
predicted_word = index_words[predicted_index[0]]

print(f"Từ dự đoán là: '{predicted_word}' với xác suất {predicted[0][predicted_index[0]]*100:.2f}%")

Ở đây kết quả mình ra được là:  
> Từ dự đoán là: **'light'** với xác suất **20.35%** ✨  

Mọi người có thể ra kết quả khác nha, điều này phụ thuộc vào trọng số của mô hình mà trọng số của mỗi mô hình khi khởi tạo là ngẫu nhiên, không giống nhau nên kết quả cuối cũng vậy. Miễn sao ý nghĩa câu văn hay các từ có phần liên kết nhau hợp lý là ok rồi. 🌟  

Tụi mình sẽ dự đoán tiếp với cụm **“sweet jeremy saw dublin”**. 🚀

In [ ]:
# Dự đoán một chuỗi
seed_text = "sweet jeremy saw dublin"
num_words = 10
predicted_text = seed_text
for i in range(num_words):
  token_list = tokenizer.texts_to_sequences([predicted_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  predicted = model1.predict(token_list, verbose=1)
  predicted_index = np.argmax(predicted, axis=-1)
  output_word = index_words[predicted_index[0]]

  print(f"Step {i}:")
  print(f"Văn bản đầu vào: {predicted_text}")
  print(f"Chuỗi mã hóa đầu vào: {token_list[0]}")
  print(f"Từ dự đoán: {output_word} với xác suất {predicted[0][predicted_index[0]]*100:2f}%")
  print("-"*50)
  predicted_text += " " + output_word


In [ ]:
print(f"Văn bản sinh ra: '{predicted_text}'")

Đây là kết quả ở lần chạy của mình:  
> Văn bản sinh ra: 'sweet jeremy saw dublin ever make a begging i neer could you dead and'✨  

Có vẻ là chúng đã ổn hơn rồi ha, sai sót thì tất nhiên vẫn có nhưng ngữ nghĩa cấu trúc đã ổn hơn tí rồi. 🌟 Bây giờ tụi mình sẽ đi thử thêm nhiều cách nữa để tăng độ hiệu quả lên hơn nha. 🚀  

# Changing the Modle Architecture
### Trước mắt thì tụi mình sẽ thử thay đổi kiến trúc mô hình xem sao.

In [ ]:
model2= Sequential([
    Embedding(total_words, 8),
    Bidirectional(LSTM(max_sequence_len-1, return_sequences=True)),
    Bidirectional(LSTM(max_sequence_len-1)),
    Dense(total_words, activation = 'softmax')
])

model2.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# Tiến hành huấn luyện
start_time = time.time()
history2 = model2.fit(xs, labels, epochs = 1000)
end_time = time.time()

In [ ]:
print(f"Thời gian huấn luyện mô hình: {timedelta(seconds=end_time - start_time)}")

In [ ]:
# Vẽ biểu đồ quá trình huấn luyện
train_acc = history2.history["accuracy"]
train_loss = history2.history["loss"]

fig, axs = plt.subplots(1, 2, figsize = (12, 4))
axs = axs.flatten()
axs[0].plot(train_acc)
axs[0].set_title("Training Accuracy")
axs[1].plot(train_loss, color="orange")
axs[1].set_title("Training Loss")
plt.tight_layout()
plt.show()

In [ ]:
# Đánh giá mô hình
model2.evaluate(xs, labels)

In [ ]:
model2.save("model2.h5")

In [ ]:
# Tải lại mô hình
model2 = load_model("model2.h5")

Tụi mình sẽ tiến hành dự đoán lại với các câu trước đó

In [ ]:
# Dự đoán một từ
seed_text = "in the town of athy"
token_list = tokenizer.texts_to_sequences([seed_text])[0]
token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre') # -1 vào max_sequence_len bởi vì chuỗi này không có nhãn
# Tiến hành dự đoán
predicted = model2.predict(token_list, verbose=1)
predicted_index = np.argmax(predicted, axis = -1)
predicted_word = index_words[predicted_index[0]]

print(f"Từ dự đoán là: '{predicted_word}' với xác suất {predicted[0][predicted_index[0]]*100:.2f}%")

In [ ]:
# Dự đoán một từ
seed_text = "sweet jeremy saw dublin"
num_words = 10
predicted_text = seed_text
for i in range(num_words):
  token_list = tokenizer.texts_to_sequences([predicted_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  predicted = model2.predict(token_list, verbose=1)
  predicted_index = np.argmax(predicted, axis=-1)
  output_word = index_words[predicted_index[0]]

  print(f"Step {i}:")
  print(f"Văn bản đầu vào: {predicted_text}")
  print(f"Chuỗi mã hóa đầu vào: {token_list[0]}")
  print(f"Từ dự đoán: {output_word} với xác suất {predicted[0][predicted_index[0]]*100:2f}%")
  print("-"*50)
  predicted_text += " " + output_word


In [ ]:
print(f"Văn bản sinh ra: '{predicted_text}'")

# Improving the Data  
### Bây giờ tụi mình tiến hành thử cách cải thiện dữ liệu đầu vào để cải tiến mô hình nha, tăng độ hiệu quả nha. ✨  

Sở dĩ mọi người thấy đường **train_accuracy** nhiễu vậy là vì dữ liệu vẫn thiếu đi phần nào đó sự liên kết ngữ cảnh mạch lạc. 🌐  

> **Vậy nguyên nhân do đâu mà ra?**  

Chính là do cách tiếp cận, chiến lược chia, xử lý dữ liệu của chúng ta trước đó. Trong một bài hát thì các câu hát đều có sự liên kết ngữ cảnh với nhau, đó cũng là cách mà con người tiếp thu. 🎶 Tuy nhiên khi đưa dữ liệu vào huấn luyện cho mô hình, chúng ta đã coi **mỗi câu hát là một dòng độc lập** và tiến hành chia chuỗi con. Chính điều này đã làm cho các câu hát trở nên mất kết nối ngữ cảnh với nhau. 🔗  

> **Ok, xác định được gốc vấn đề rồi vậy chúng ta sẽ làm gì tiếp theo đây?**  

Câu trả lời là làm sao cho các chuỗi văn bản huấn luyện có chứa một phần của câu trước và câu sau, từ đó giữa các câu sẽ có sự liên kết với nhau. 🧩  

Bây giờ thay vì để các câu trong bài hát thành một hàng độc lập, chúng ta sẽ tiến hành **gom tất cả các câu đó lại về chung một hàng. Chúng ta sẽ không chia theo ký tự xuống hàng "\n" nữa mà là theo dấu khoảng cách**  🚀  

**Vậy workflow của chúng ta sẽ như sau:**

**1. Chia các câu dựa trên cửa sổ trượt:** Chia đoạn văn thành danh sách các từ, sau đó dùng cửa sổ trượt để tạo ra các câu hay chuỗi con. Qua đó ta có thể giữ được sự liên kết ngữ cảnh của các câu với nhau.

**2. Tiến hành chia các chuỗi con từ các chuỗi đã chia trước đó:** làm giống như cách ban đâu tuy nhiên thay vì dữ liệu của chúng ta là các câu độc lập thì giờ là các câu đã được tạo ra ở bước 1.k

In [ ]:
# Kiểm tra lại dữ liệu
print(repr(data))

Tiến hành chia các câu, chuỗi dựa trên cửa sổ trượt

In [ ]:
import regex as re
# Chia theo khoảng cách thay vì kí tự xuống hàng "\n"
window_size = 6

# Tiến hành lấy chuôi các từ trong văn bản
corpus = data.lower()
words = corpus.split(" ")

# Tiến hành chia các câu (chuỗi) dựa trên cửa sổ trượt
range_size = len(words) - window_size + 1

input_sentences = []
for i in range(range_size):
  input_sentences.append(" ".join(words[i:i+window_size]))


# Kiểm tra 5 câu (chuỗi) đầu tiên
for i in range(5):
  print(repr(input_sentences[i]))

**Các câu ở chiến lược chia dữ liệu cũ:**
>come all ye maidens young and fair   
and you that are blooming in your prime   
always beware and keep your garden fair   
let no man steal away your thyme   
for thyme it is a precious thing   


**Các câu chia theo phương pháp cửa sổ trượt:**
>come all ye maidens young and   
all ye maidens young and fair\nand   
ye maidens young and fair\nand you   
maidens young and fair\nand you that    
young and fair\nand you that are

Bây giờ tụi mình đến với bước 2, tiến hành mã hóa và chia chuỗi con nha.

In [ ]:
# Tạo tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_sentences)

sub_sequences = []
# Tiến hành mã hóa và chia chuỗi con
for sentence in input_sentences:
  token_list = tokenizer.texts_to_sequences([sentence])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    sub_sequences.append(n_gram_sequence)

# Kiểm tra 5 chuỗi đầu tiên
for i in range(5):
  print(sub_sequences[i])

In [ ]:
# Kiểm tra kích thước từ điển
print(f"Tổng số lượng từ: {len(tokenizer.word_index)}")
print(f"Bộ từ điển từ vựng: {tokenizer.word_index}")

In [ ]:
# Tạo từ điển chuyển số thành từ
index_words ={}
for word, index in tokenizer.word_index.items():
    index_words[index] = word

Ở đây có một lưu ý nhỏ: vì bản thân tokenizer đã có sẵn một bộ lọc để các kí tự đặc biệt nên các từ nối liền bởi kí tự đặc biệt (ví dụ: "mother-in-law" sẽ bị chuyển thành "mother in law"). Do đó sẽ có một số chuỗi có kích thước lớn hơn so với cửa sổ trượt. Ta sẽ tiến hành tính lại độ dài chuỗi dài nhất.

In [ ]:
max_sequence_len = max([len(x) for x in sub_sequences])
print(f"Độ dài chuỗi dài nhất: {max_sequence_len}")

Kết quả là:
> Độ dài chuỗi dài nhất: 22

Như chúng mình đã dự đoán nhưng không ngờ kích thước nó có thể tăng lên nhiều vậy.

In [ ]:
# Kiểm tra 5 chuỗi có độ dài lớn hơn cửa sổ trượt.
sequences_higher_than_window = []
for i in range(len(sub_sequences)):
  if len(sub_sequences[i]) > window_size:
    sequences_higher_than_window.append(sub_sequences[i])

for i in range(5):
  print(sequences_higher_than_window[i])

Bây giờ tụi mình sẽ tiến hành padding để tất cả chuỗi có cùng kích thước, độ dài nha.

In [ ]:
input_sequences = np.array(pad_sequences(sub_sequences, maxlen=max_sequence_len, padding="pre"))

# Chia dữ liệu x và labels để huấn luyện.
xs = input_sequences[:,:-1]
labels = input_sequences[:,-1]

Tiến hành tạo dataset để tối ưu hóa dữ liệu đẩy vào quá trình huấn luyện

In [ ]:
# Tạo datagen để xủ lý nhanh, tối ưu hóa quá trình huấn luyện hơn
# Tạo đối tượng Dataset
dataset = tf.data.Dataset.from_tensor_slices((xs, labels))

# Tối ưu Dataset với shuffle, batch và prefetch
AUTOTUNE = tf.data.AUTOTUNE
dataset = (
    dataset
    .cache("train_cache.tfdata")    # Thay vì lưu dữ liệu vào ram thì lưu hẳn vào disk
    .shuffle(buffer_size=1000)      # Trộn dữ liệu
    .batch(32)                      # Chia batch (batch_size=32)
    .prefetch(buffer_size=AUTOTUNE) # Tải trước dữ liệu
)

Mình sẽ tiến hành định nghĩa kiến trúc mô hình theo đúng lần thử của tác giả, mọi người có thể linh hoạt thay đổi các siêu tham số liên tục để cải thiện nha.  

Tác giả sau khi thử nhiều lần thì có đề xuất:  
- **window_size**: 6  
- **số chiều embedding**: 16  
- **số unit trong LSTMs**: 32  
- **tăng tốc độ học-learning rate lên**: tác giả không nói rõ nên mình tạm lấy **0.005** đi. Số mặc định là **0.001**  
- **epochs**: 100 bởi số lượng tham số tính toán này là rất lớn. 🚀  

In [ ]:
total_words = len(tokenizer.word_index) + 1 # vì từ điển bắt đầu từ 1, số 0 được thêm vào như padding

In [ ]:
with strategy.scope():
  model3 = Sequential([
      Embedding(total_words, 16),
      Bidirectional(LSTM(32, return_sequences=True)),
      Bidirectional(LSTM(32)),
      Dense(total_words, activation="softmax")
  ])
  adam = Adam(learning_rate=0.01)
  model3.compile(loss = 'sparse_categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

In [ ]:
start_time = time.time()
history3 = model3.fit(dataset, epochs = 100)
end_time = time.time()

In [ ]:
print(f"Thời gian huấn luyện mô hình: {timedelta(seconds=end_time - start_time)}")

In [ ]:
# Vẽ biểu đồ huấn luyện
train_acc = history3.history["accuracy"]
train_loss = history3.history["loss"]

fig, axs = plt.subplots(1, 2, figsize = (12, 4))
axs = axs.flatten()
axs[0].plot(train_acc)
axs[0].set_title("Training Accuracy")
axs[1].plot(train_loss, color="orange")
axs[1].set_title("Training Loss")
plt.tight_layout()
plt.show()

In [ ]:
# Đánh giá mô hình
model3.evaluate(dataset)

In [ ]:
model3.save("model3.h5")

In [ ]:
# Tải lại mô hình
model3 = load_model("model3.h5")

In [ ]:
# Tiến hành dự đoán
seed_text = "sweet jeremy saw dublin"
num_words = 100
predicted_text = seed_text

# Vì kích thước đầu vào đã thay đổi theo kích thước cửa sổ
# nên ta cũng thay đổi phần cắt dữ liệu đưa vào tí
for i in range(num_words):
  # Kích thước cửa sổ cắt ra ban đầu là 6 nhưng phải chia từ cuối cho nhãn nên đầu vào là 5
  token_list = tokenizer.texts_to_sequences([predicted_text])[0]
  # Tiến hành kiểm tra luôn nếu kích thước không đủ thì padding vào hoặc dài quá thì có thể cắt đi
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding="pre")
  token_list = np.array(token_list)
  predicted_result = model3.predict(token_list)
  predicted_index = np.argmax(predicted_result, axis=-1)
  output_word = index_words.get(predicted_index[0], "") # Nếu index không có trong từ điển, trả về kí tự rỗng như padding

  print(f"Step {i}:")
  print(f"Từ dự đoán: {output_word} với xác suất {predicted_result[0][predicted_index[0]]*100:2f}%")
  print("-"*50)
  predicted_text += " " + output_word


In [ ]:
print(f"Văn bản sinh ra: \n{predicted_text}")

# Character-Based Encoding  
### Tụi mình sẽ tiếp cận với việc mã hóa ở cấp kí tự nha.  

Nếu như ở các chương trước chúng ta thường hay sử dụng mã hóa ở cấp từ thì ở chương này chúng ta cần phải xem xét lại. Với một lượng văn bản khổng lồ như này, kích thước bộ từ điển từ cũng sẽ rất là lớn. Khi đến với tác vụ tạo sinh văn bản, chẳng phải đầu của mô hình bằng với kích thước bộ từ điển và nó thật sự là một con số choáng ngợp.  

Trong khi đó nếu ta sử dụng mã hóa ở cấp kí tự thì kích thước bộ từ điển sẽ nhỏ hơn rất nhiều, đầu ra mà mô hình cũng ít hơn. Thậm chí thấp hơn cả trăm lần. ✨  

**VD**: Chúng ta có một bộ dữ liệu văn bản:  
- Khi mã hóa ở cấp từ, bộ từ điển của chúng ta có kích thước là **2700** tương ứng với 2700 từ riêng biệt.  
- Tuy nhiên khi mã hóa ở cấp kí tự thì kích thước từ điển chỉ còn **65**, nhỏ hơn rất là nhiều, mô hình cũng chỉ cần dự đoán đầu ra là **65 nhãn** thay vì **2700** như trước.  

Qua cách mã hóa dữ liệu trên thì mô hình của chúng ta sẽ đơn giản hơn rất nhiều, ngoài ra cũng có thể chứa các kí tự đặc biệt. 🚀  

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kewagbln/shakespeareonline")

print("Path to dataset files:", path)

In [ ]:
# Liệt kê danh sách file
for f in os.listdir(path):
  print(f)

In [ ]:
# Load dữ liệu vào
data = open(os.path.join(path,"t8.shakespeare.txt"), "r").read()
# Kiểm tra dữ liệu
print(data)

In [ ]:
# Tiến hành chia câu dựa trên cửa sổ trượt
window_size = 17
corpus = data.lower()

# Lấy danh sách kí tự
characters = list(corpus)
# print(characters)

# Chia câu (chuỗi) dựa trên cửa sổ trượt
range_size = len(characters) - window_size + 1
input_sentences = []
for i in range(range_size):
  input_sentences.append("".join(characters[i:i+window_size]))

# Kiểm tra 5 câu (chuỗi) đầu tiên:
for i in range(5):
  print(repr(input_sentences[i]))

In [ ]:
import pickle

# Tiến hành tạo bộ mã hóa cấp kí tự và chia chuỗi con
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(input_sentences)


In [ ]:
print(f"Tổng số lượng kí tự: {len(tokenizer.word_index)}")
print(f"Bộ từ điển từ vựng: {tokenizer.word_index}")

In [ ]:
# Tạo bộ từ điển chuyển đổi
index_words = {}
for word, index in tokenizer.word_index.items():
  index_words[index] = word

Vì vấn đề ram không đủ, lượng dữ liệu quá lớn nên mình tiến hành lưu từng dữ liệu mã hóa vào một file pickle nha.

In [ ]:

# Tạo file tạm để lưu dữ liệu
temp_file = "sub_sequences_temp.pkl"
max_sequence_len = 0

with open(temp_file, "wb") as f:
  for sentence in input_sentences:
    token_list = tokenizer.texts_to_sequences([sentence])[0]
    if len(token_list) > max_sequence_len:
      max_sequence_len = len(token_list)
    for i in range(1, len(token_list)):
      n_gram_sequence = token_list[:i+1]
      pickle.dump(n_gram_sequence, f)

print(f"Độ dài chuỗi dài nhất: {max_sequence_len}")

 Sau khi tiến hành lưu một file pickle xong, ta sẽ tiến hành xử lý theo từng batch, tuy là việc này sẽ tốn thời gian hơn rất nhiều nhưng đây là cách duy nhất vì lượng dữ liệu quá lớn.

In [ ]:
temp_file = "sub_sequences_temp.pkl"
def read_sequences_in_batches(file_path, batch_size):
    """
    Hàm generator đọc dữ liệu từ file Pickle theo từng batch.
    """
    with open(file_path, "rb") as f:
        batch = []
        while True:
            try:
                # Đọc từng chuỗi
                n_gram_sequence = pickle.load(f)
                batch.append(n_gram_sequence)
                # Nếu đủ batch size, yield batch và khởi tạo lại
                if len(batch) == batch_size:
                    yield batch
                    batch = []
            except EOFError:
                # Trả về batch cuối cùng nếu còn dữ liệu
                if batch:
                    yield batch
                break

In [ ]:
def process_batch(batch, max_seq_length):
    """
    Pad các chuỗi và tách X (input), labels (output).
    """
    # Pad các chuỗi đến chiều dài tối đa
    padded_batch = pad_sequences(batch, maxlen=max_seq_length, padding='pre')

    # Tách X và labels
    X = padded_batch[:, :-1]  # Tất cả trừ phần tử cuối
    labels = padded_batch[:, -1]  # Phần tử cuối cùng làm nhãn
    return X, labels

In [ ]:
def data_generator(file_path, batch_size, max_seq_length):
    for batch in read_sequences_in_batches(file_path, batch_size):
        X, labels = process_batch(batch, max_seq_length)
        yield X, labels

batch_size = 16
dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(temp_file, batch_size, max_sequence_len),
    output_signature=(
        tf.TensorSpec(shape=(None, max_sequence_len - 1), dtype=tf.int32),  # X
        tf.TensorSpec(shape=(None,), dtype=tf.int32)  # labels
    )
)

# Prefetch để tăng hiệu suất
dataset = dataset.cache("train_cache.tfdata").prefetch(tf.data.experimental.AUTOTUNE)



In [ ]:
# Kiểm tra số lượng kí tự tối đa
total_words = len(tokenizer.word_index) + 1
print(f"Số lượng kí tự: {total_words}")

Uầy vậy là số lượng nhãn đầu ra mà mô hình phải dự đoán đã thấp hơn rất nhiều rồi, chỉ 66 kí tự. Bây giờ tụi mình tiến hành định nghĩa và huấn luyện mô hình thử nha.

In [ ]:
# Định nghĩa mô hình
with strategy.scope():
  model4 = Sequential([
      Embedding(total_words, 16),
      Bidirectional(LSTM(32, return_sequences=True)),
      Bidirectional(LSTM(32)),
      Dense(total_words, activation="softmax")
  ])
  adam = Adam(learning_rate=0.01)
  model4.compile(loss = 'sparse_categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

In [ ]:
model4.summary()

In [ ]:
start_time = time.time()
history4 = model4.fit(dataset, epochs = 100)
end_time = time.time()

In [ ]:
print(f"Thời gian huấn luyện mô hình: {timedelta(seconds=end_time - start_time)}")

In [ ]:
# Vẽ biểu đồ quá trình huấn luyện
train_acc = history4.history["accuracy"]
train_loss = history4.history["loss"]

fig,axs = plt.subplots(1, 2, figsize = (12, 4))
axs = axs.flatten()
axs[0].plot(train_acc)
axs[0].set_title("Training Accuracy")
axs[1].plot(train_loss, color="orange")
axs[1].set_title("Training Loss")
plt.tight_layout()
plt.show()

In [ ]:
model4.eval(dataset)

In [ ]:
model4.save("model4.h5")

In [ ]:
# Cập nhật trọng số từ mô hình đã lưu
model4 = tf.keras.models.load_model('model4.h5')

In [ ]:
# Tiến hành dự đoán
seed_text = "sweet jeremy saw dublin"
num_words = 500
predicted_text = seed_text

# Vì kích thước đầu vào đã thay đổi theo kích thước cửa sổ
# nên ta cũng thay đổi phần cắt dữ liệu đưa vào tí
for i in range(num_words):
  # Kích thước cửa sổ cắt ra ban đầu là 6 nhưng phải chia từ cuối cho nhãn nên đầu vào là 5
  token_list = tokenizer.texts_to_sequences([predicted_text])[0]
  # Tiến hành kiểm tra luôn nếu kích thước không đủ thì padding vào hoặc dài quá thì có thể cắt đi
  token_list = pad_sequences([token_list], maxlen=window_size-1, padding="pre")
  token_list = np.array(token_list)
  predicted_result = model4.predict(token_list)
  predicted_index = np.argmax(predicted_result, axis=-1)
  output_char = index_words[predicted_index[0]]

  print(f"Step {i}:")
  print(f"Từ dự đoán: {output_char} với xác suất {predicted_result[0][predicted_index[0]]*100:2f}%")
  print("-"*50)
  predicted_text += " " + output_char


In [ ]:
print("Văn bản sinh ra:")
print(predicted_text)